# DReS-FL Experiment 

- K = 1
- T = 1
- input dimension = 2 -> scalar
- linear regression
- w = [1 1]
- finite field size = 5
- inputs are selected from {0, 1}
- no relationship between x and y
- the gradient is supposed to be revealed at the end of each training round

Basically, our objective function is $l = (y - xw)^2$ and our gradient is $g = -2(xy - x^2w)$ 

In [1]:
import numpy as np
from lcc.polynomials import LCCPoly, InterpolatedPoly, Poly
from mutual_information.estimators.neural.benchmark import neural_estimator_benchmark
from lcc.domain import create_lcc_gradient_domain_basic_setup_no_relationship
from mutual_information.exhaustive_search_mutual_information import calculate_mutual_information_domain

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [2]:
prime = 5
data_range = 2
para_param = 1 # K
priv_param = 1 # T

num_of_samples = 10
beta_arr = [0, 1]
alpha_arr = [2, 3, 4] # number of clients = 3
weight = 2

In [3]:
def create_encoded_dataset(encoded_data_pol, data_size, encoded_label_pol, alpha):
    encoded_data = np.empty((len(encoded_data_pol), *data_size)) 
    encoded_label = np.empty((len(encoded_label_pol), 1))
    for idx, (data_pol, label_pol) in enumerate(zip(encoded_data_pol, encoded_label_pol)):
        encoded_data[idx] = data_pol(alpha)
        encoded_label[idx][1] = label_pol(alpha)
    return encoded_data, encoded_label

def calculate_gradient(encoded_data, encoded_label, curr_weight, p):
    encoded_data = encoded_dataset[:, 0].reshape(-1, 1)
    encoded_label = encoded_dataset[:, 1].reshape(-1, 1)
    
    gradient = (encoded_label - encoded_data @ curr_weight) % p
    gradient = (encoded_data.T @ gradient) % p
    gradient = (-2 * gradient) % p
    return gradient

def calculate_gradient_samplewise(encoded_dataset, curr_weight, p):
    # g = -2(xy - x^2w)
    gradient = np.empty(encoded_dataset.shape[0])
    for idx, encoded_sample in enumerate(encoded_dataset):
        data, label = encoded_sample[0], encoded_sample[1]
        gradient[idx] = (label - data * curr_weight) % p
        gradient[idx] = (data * gradient[idx]) % p
        gradient[idx] = (-2 * gradient[idx]) % p
    gradient = gradient.astype(int)
    return gradient

In [4]:
secret_data = np.random.randint(low=0, high=2, size=(num_of_samples, 2, 1))
secret_label = np.random.randint(low=0, high=2, size=(num_of_samples, 1))
encoded_secret_data_pol = [LCCPoly(beta_arr, [x], para_param, priv_param, prime, size=(2, 1)) for x in secret_data]
encoded_secret_label_pol = [LCCPoly(beta_arr, [x[0]], para_param, priv_param, prime) for x in secret_label]

In [5]:
# client 0
client_0_encoded_data = create_encoded_dataset(encoded_secret_data_pol, encoded_secret_label_pol, alpha_arr[0])
client_0_encoded_gradient = calculate_gradient_samplewise(client_0_encoded_data, weight, prime)

# client 1
client_1_encoded_data = create_encoded_dataset(encoded_secret_data_pol, encoded_secret_label_pol, alpha_arr[1])
client_1_encoded_gradient = calculate_gradient_samplewise(client_1_encoded_data, weight, prime)

# client 2
client_2_encoded_data = create_encoded_dataset(encoded_secret_data_pol, encoded_secret_label_pol, alpha_arr[2])
client_2_encoded_gradient = calculate_gradient_samplewise(client_2_encoded_data, weight, prime)

In [6]:
revealed_poly_arr = [] 
revealed_gradients = np.empty((client_0_encoded_gradient.shape[0], 1))
revealed_random = np.empty((client_0_encoded_gradient.shape[0], 1))
revealed_poly_constructed_coeff = np.empty((client_0_encoded_gradient.shape[0], 2 * para_param + 1))
for gradient_idx in range(client_0_encoded_gradient.shape[0]):
    revealed_poly = InterpolatedPoly([int(client_0_encoded_gradient[gradient_idx]), int(client_1_encoded_gradient[gradient_idx]), int(client_2_encoded_gradient[gradient_idx])], alpha_arr, prime)
    revealed_gradients[gradient_idx][0] = revealed_poly(beta_arr[0])
    revealed_random[gradient_idx][0] = revealed_poly(beta_arr[1])
    revealed_poly_constructed_coeff[gradient_idx] = revealed_poly.coefficients
    revealed_poly_arr.append(revealed_poly)
    

In [7]:
# create dataset
resulting_dataset = np.concatenate([secret_data, secret_label, revealed_gradients, revealed_random, revealed_poly_constructed_coeff], axis=1)

In [10]:
estimators_all, results_all = neural_estimator_benchmark(resulting_dataset[:, :2], resulting_dataset[:, 2:])

 22%|███████████▉                                         | 2250/10000 [03:43<12:50, 10.06step/s, test=1.29, train=1.28]


Donker Varadhan estimator: 1.29


  8%|███▉                                                 | 750/10000 [00:03<00:38, 238.42step/s, test=1.29, train=1.31]


MINE estimator: 1.32


 18%|█████████▎                                           | 1750/10000 [02:54<13:44, 10.00step/s, test=1.29, train=1.31]


InfoNCE estimator: 1.29


 15%|███████▉                                             | 1500/10000 [02:29<14:08, 10.02step/s, test=1.29, train=1.30]

NWJ estimator: 1.29


In [11]:
estimators_revealed_gradient, results_revealed_gradient = neural_estimator_benchmark(resulting_dataset[:, :2], resulting_dataset[:, 2].reshape(-1, 1))

 15%|███████▉                                             | 1500/10000 [03:26<19:29,  7.27step/s, test=1.04, train=1.06]


Donker Varadhan estimator: 1.04


 10%|█████▏                                              | 1000/10000 [00:03<00:34, 261.57step/s, test=1.03, train=1.12]


MINE estimator: 1.06


  8%|████                                                  | 750/10000 [01:41<20:54,  7.38step/s, test=1.04, train=1.03]


InfoNCE estimator: 1.04


 15%|███████▉                                             | 1500/10000 [03:33<20:07,  7.04step/s, test=1.04, train=1.06]

NWJ estimator: 1.04


In [12]:
basic_experiment_domain = create_lcc_gradient_domain_basic_setup_no_relationship(data_range, beta_arr, alpha_arr, weight, prime)
all_coeff_mi = calculate_mutual_information_domain(basic_experiment_domain, [0, 2], [4, 5, 6, 7, 8], [data_range, data_range], [prime, prime, prime, prime, prime])
revealed_gradient_mi = calculate_mutual_information_domain(basic_experiment_domain, [0, 2], [4], [data_range, data_range], [prime])
print(all_coeff_mi, revealed_gradient_mi)

iteration 100/12500
iteration 200/12500
iteration 300/12500
iteration 400/12500
iteration 500/12500
iteration 600/12500
iteration 700/12500
iteration 800/12500
iteration 900/12500
iteration 1000/12500
iteration 1100/12500
iteration 1200/12500
iteration 1300/12500
iteration 1400/12500
iteration 1500/12500
iteration 1600/12500
iteration 1700/12500
iteration 1800/12500
iteration 1900/12500
iteration 2000/12500
iteration 2100/12500
iteration 2200/12500
iteration 2300/12500
iteration 2400/12500
iteration 2500/12500
iteration 2600/12500
iteration 2700/12500
iteration 2800/12500
iteration 2900/12500
iteration 3000/12500
iteration 3100/12500
iteration 3200/12500
iteration 3300/12500
iteration 3400/12500
iteration 3500/12500
iteration 3600/12500
iteration 3700/12500
iteration 3800/12500
iteration 3900/12500
iteration 4000/12500
iteration 4100/12500
iteration 4200/12500
iteration 4300/12500
iteration 4400/12500
iteration 4500/12500
iteration 4600/12500
iteration 4700/12500
iteration 4800/12500
i